In [1]:
from digital_manuscript import BnF
manuscript = BnF()

In [2]:
import pandas as pd
df = pd.DataFrame(columns=['entry_id', 'version', 'position', 'render', 'length'])
i = 0
for identity, entry in manuscript.entries.items():
    if len(entry.margins) > 0:
        for version, margin_list in entry.margins.items():
            for margin in margin_list:
                df.loc[i] = [identity, version, margin.position, margin.render, margin.length]
                i += 1
df.to_csv('margins.csv', index=False)
            
            

In [3]:
import re

def find_percent_marginal(entry, version):
    if len(entry.margins) == 0:
        return 0

    margin_sum = 0
    for margin in entry.margins[version]:
        margin_sum += margin.length
    
    return 100 * margin_sum/entry.length[version]

def find_percent_continued(entry, version):
    continued_sum = 0
    parts = re.findall(r'(<div([\w\s=";-]*)>(.*?)</div>)', re.sub(r'\s+', ' ', entry.text(version, True)))
    for part in parts:
        _, attributes, text = part
        if 'continues="yes"' in attributes:
                text = re.sub(r'<.*?>', '', text)
                continued_sum += len(text)
    return 100 * continued_sum / entry.length[version]
                

In [53]:
df = pd.DataFrame(columns=['entry_id', 'length_tc', 'length_tcn', 'length_tl',
                           'percent_marginal_tc', 'percent_marginal_tcn', 'percent_marginal_tl',
                           'percent_continued_tc', 'percent_continued_tcn', 'percent_continued_tl'])
versions = ['tc', 'tcn', 'tl']
i=0

for identity, entry in manuscript.entries.items():
    if len(entry.margins) > 0 or 'continues="yes"' in entry.text('tl', xml=True):
        percent_marginal, percent_continued = {}, {}
        for version in versions:
            percent_marginal[version] = find_percent_marginal(entry, version)
            percent_continued[version] = find_percent_continued(entry, version)

        df.loc[i] = [entry.identity, entry.length['tc'], entry.length['tcn'], entry.length['tl'],
                    percent_marginal['tc'], percent_marginal['tcn'], percent_marginal['tl'],
                    percent_continued['tc'], percent_continued['tcn'], percent_continued['tl']]
        i += 1
df
            
        

,entry_id,length_tc,length_tcn,length_tl,percent_marginal_tc,percent_marginal_tcn,percent_marginal_tl,percent_continued_tc,percent_continued_tcn,percent_continued_tl
0,001r_1,292,302,299,95.205479,95.033113,94.983278,0.0,0.0,0.0
1,001r_2,133,140,131,96.240602,95.000000,96.183206,0.0,0.0,0.0
2,001r_3,1502,1513,1511,97.936085,97.951091,98.146923,0.0,0.0,0.0
3,001r_4,94,98,96,81.914894,82.653061,84.375000,0.0,0.0,0.0
4,001v_1,392,406,372,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
924,170r_6,631,647,580,35.340729,35.703246,36.724138,0.0,0.0,0.0
925,170v_1,498,507,523,0.000000,0.000000,0.000000,0.0,0.0,0.0
926,170v_2,269,278,282,16.728625,15.827338,15.957447,0.0,0.0,0.0
927,170v_3,497,510,536,12.273642,11.764706,11.567164,0.0,0.0,0.0
